<hr style="border:0.1px solid gray">

# <center style='color:Red'> COSC2753 - Machine Learning </center> 
## <center style='color:Red'> Assignment 2 - Classify Images of Colon Cancer </center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#-COSC2753---Machine-Learning-" data-toc-modified-id="-COSC2753---Machine-Learning--1"><center style="color:Red"> COSC2753 - Machine Learning </center></a></span><ul class="toc-item"><li><span><a href="#-Assignment-2---Classify-Images-of-Colon-Cancer-" data-toc-modified-id="-Assignment-2---Classify-Images-of-Colon-Cancer--1.1"><center style="color:Red"> Assignment 2 - Classify Images of Colon Cancer </center></a></span></li></ul></li><li><span><a href="#Main-Task" data-toc-modified-id="Main-Task-2">Main Task</a></span></li><li><span><a href="#1.-Import-libraries-and-setting-up-environment" data-toc-modified-id="1.-Import-libraries-and-setting-up-environment-3">1. Import libraries and setting up environment</a></span></li><li><span><a href="#2.-Import-Dataset" data-toc-modified-id="2.-Import-Dataset-4">2. Import Dataset</a></span></li><li><span><a href="#3.-Exploratory-Data-Analysis-(EDA)" data-toc-modified-id="3.-Exploratory-Data-Analysis-(EDA)-5">3. Exploratory Data Analysis (EDA)</a></span><ul class="toc-item"><li><span><a href="#3.1.-Process-the-data" data-toc-modified-id="3.1.-Process-the-data-5.1">3.1. Process the data</a></span></li><li><span><a href="#3.2.-Data-Analysis-and-Visualiztion" data-toc-modified-id="3.2.-Data-Analysis-and-Visualiztion-5.2">3.2. Data Analysis and Visualiztion</a></span><ul class="toc-item"><li><span><a href="#3.2.1.-Correlation-and-Correlation-Matrix" data-toc-modified-id="3.2.1.-Correlation-and-Correlation-Matrix-5.2.1">3.2.1. Correlation and Correlation Matrix</a></span></li><li><span><a href="#3.2.2.-Data-Distribution" data-toc-modified-id="3.2.2.-Data-Distribution-5.2.2">3.2.2. Data Distribution</a></span></li><li><span><a href="#Countplot" data-toc-modified-id="Countplot-5.2.3">Countplot</a></span></li><li><span><a href="#Histogram" data-toc-modified-id="Histogram-5.2.4">Histogram</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-5.2.5">Conclusion</a></span></li></ul></li><li><span><a href="#3.3.-Data-Preparation" data-toc-modified-id="3.3.-Data-Preparation-5.3">3.3. Data Preparation</a></span></li><li><span><a href="#Image-functions" data-toc-modified-id="Image-functions-5.4">Image functions</a></span></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------5.5">------------------------</a></span></li></ul></li><li><span><a href="#4.-Models" data-toc-modified-id="4.-Models-6">4. Models</a></span><ul class="toc-item"><li><span><a href="#4.1-Convolutional-neural-network" data-toc-modified-id="4.1-Convolutional-neural-network-6.1">4.1 Convolutional neural network</a></span><ul class="toc-item"><li><span><a href="#4.1.1-Baseline-model" data-toc-modified-id="4.1.1-Baseline-model-6.1.1">4.1.1 Baseline model</a></span></li></ul></li></ul></li><li><span><a href="#summarize-history-for-accuracy" data-toc-modified-id="summarize-history-for-accuracy-7">summarize history for accuracy</a></span></li><li><span><a href="#summarize-history-for-loss" data-toc-modified-id="summarize-history-for-loss-8">summarize history for loss</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#4.1.2-Hyperparameter-tuning" data-toc-modified-id="4.1.2-Hyperparameter-tuning-8.0.1">4.1.2 Hyperparameter tuning</a></span><ul class="toc-item"><li><span><a href="#I.-Layers-tuning" data-toc-modified-id="I.-Layers-tuning-8.0.1.1">I. Layers tuning</a></span></li></ul></li><li><span><a href="#4.1.3-Final-model" data-toc-modified-id="4.1.3-Final-model-8.0.2">4.1.3 Final model</a></span></li></ul></li></ul></li></ul></div>

**Group Number:** 8

**Student 1:** Ngo Quang Khai - s3836387 

**Student 2:** Nguyen Quoc Minh - s3758994

**Student 3:** Bui Thanh Huy - s3740934

**Student 4:** Tran Vinh Khang - s3855823

**Student 5:** Le Trong Hung - s3805504

**Lecturer:** Dr. Bao Nguyen

**Due Date:** May 14th, 2021

<hr style="border:0.1px solid gray">

# Main Task

- **Task 1**: Classify images according to whether given cell image represents a cancerous cells or not (isCancerous)


- **Task 2**: Classify images according to cell-type, such as: fibroblast, inflammatory, epithelial or others.


- **Extra task**:

# 1. Import libraries and setting up environment

In [2]:
# !pip install tensorflow
# !pip install keras
# !pip install opencv-python
# !pip install bayesian-optimization 

In [2]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn as sk
import seaborn as sns
import cv2
import os
import tensorflow as tf

from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import RandomOverSampler

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.regularizers import l2
from keras import backend as K
from math import floor
from sklearn.metrics import make_scorer, accuracy_score, classification_report

from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold,GridSearchCV

from PIL import Image
import warnings
warnings.filterwarnings('ignore')

LeakyReLU = LeakyReLU(alpha=0.1)

In [3]:
#It is important to check the version of these libraries to make sure that implement the latest method.
print('Tensorflow version:', tf.__version__)
print('CV2 version:', cv2.__version__)

Tensorflow version: 2.6.0
CV2 version: 4.5.5


# 2. Import Dataset

In [4]:
# Import data_labels_mainData.csv into a DataFrame
main_data = pd.read_csv('data_labels_mainData.csv')

# Import data_labels_extraData.csv into a DataFrame
extra_data = pd.read_csv('data_labels_extraData.csv')

# 3. Exploratory Data Analysis (EDA)

## 3.1. Process the data

In [5]:
#See the data frame of main_data
print("Dataframe of main_data: \n", main_data, "\n")

#See the data frame of extra_data
print("Dataframe of extra_data: \n", extra_data)

Dataframe of main_data: 
       InstanceID  patientID  ImageName cellTypeName  cellType  isCancerous
0          22405          1  22405.png   fibroblast         0            0
1          22406          1  22406.png   fibroblast         0            0
2          22407          1  22407.png   fibroblast         0            0
3          22408          1  22408.png   fibroblast         0            0
4          22409          1  22409.png   fibroblast         0            0
...          ...        ...        ...          ...       ...          ...
9891        1625         60   1625.png   epithelial         2            1
9892        1626         60   1626.png   epithelial         2            1
9893        1627         60   1627.png   epithelial         2            1
9894        1628         60   1628.png   epithelial         2            1
9895        1629         60   1629.png   epithelial         2            1

[9896 rows x 6 columns] 

Dataframe of extra_data: 
        InstanceID  p

**=>** Next, we will check if the data have any null values and datatype of each columns.

In [7]:
main_data.info()
main_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9896 entries, 0 to 9895
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   InstanceID    9896 non-null   int64 
 1   patientID     9896 non-null   int64 
 2   ImageName     9896 non-null   object
 3   cellTypeName  9896 non-null   object
 4   cellType      9896 non-null   int64 
 5   isCancerous   9896 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 464.0+ KB


InstanceID      0
patientID       0
ImageName       0
cellTypeName    0
cellType        0
isCancerous     0
dtype: int64

In [8]:
extra_data.info()
extra_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10384 entries, 0 to 10383
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   InstanceID   10384 non-null  int64 
 1   patientID    10384 non-null  int64 
 2   ImageName    10384 non-null  object
 3   isCancerous  10384 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 324.6+ KB


InstanceID     0
patientID      0
ImageName      0
isCancerous    0
dtype: int64

**=>** Since our data is no missing/null value and properly formatted, we will move on to analyzing data.

## 3.2. Data Analysis and Visualiztion

Based on the information provided by the biomedical company as well as the DataFrames in **section 3.1** above. We are aware of:
- The **main_data** contains both **cellType (0 to 3)** and **isCancerous (0 and 1)** columns. In there, **cellType** values 0 to 3 represent fibroblast, inflammatory, epithelial, and miscellaneous cell types respectively based on the **cellTypeName** column. In addition, **isCancerous** values 0 and 1 represent colon cells with non-Cancer and Cancer.


- The **extra_data** only contains **isCancerous (0 and 1)** column.

**=>** Therefore, we choose the **isCancerous** attribute as the target column because both **main_data** and **extra_data** have the column **isCancerous**.

### 3.2.1. Correlation and Correlation Matrix

Firstly, we will draw the correlation matrix to find out any relationship between **isCancerous** column with the others
columns in **main_data**.

**Analysis**

As we can see from the matrix above, most of the attributes do not correlate with **isCancerous** much are **InstanceID** (-0.44), **patientID** (-0.0012). Since these attributes do not affect much to isCancerous attribute, we will not use these attributes to train our model. Applying these two attributes to train the model can cause noise and overcomplicated issue to our prediction model.

**=>** There is only attribute **cellType** with 0.44 has high impact on the attribute 'isCancerous'

### 3.2.2. Data Distribution

### Countplot

**Analysis**
- The first countplot shows us there 4 different types of cell: 0, 1, 2, 3. The first type of cell is 0 has approximately 1900, the second type of cell is 2 has approximately 2600, the third one is 4100 and last one is 1500.


- The second countplot shows us number of cells that are cancer and not. In the first column is 0 represent the cell that are not cancerous, and 1 represent the cell that are cancerous. There are approximately 5900 cells that are not cancerous and 4200 cells that are cancerous in our data.

### Histogram

**Analysis**

Looking at these two histograms, we can see that 'cellType' 0, 1, and 3 are not cancerous. Meanwhile only 'cellType' 2 is cancerous.

### Conclusion

- In **main_data**, only the attribute **cellType** continue to be our strongest candidate for to influence our model when it has high impact on our target attribute is **isCancerous**.


- Attributes such as **InstanceID** and **patientID** might create a lot of noise in our model. 


- There are 4 types of cells: 0, 1, 2, 3. Only cell type number 2 is cancerous while others are not cancerous.


- We expect the the model would predict class 0 more accurately than class 1

## 3.3. Data Preparation

In [6]:

traindata, validata = train_test_split(main_data[['ImageName','cellType']], 
                                              test_size=0.3, random_state=9, stratify = main_data[['cellType']])                                        

print('Training data shape task 1:', traindata.shape)
print('Validation data shape task 1:', validata.shape)


Training data shape task 1: (6927, 2)
Validation data shape task 1: (2969, 2)


In [13]:
print(traindata['ImageName'])

1242     4988.png
4946    18534.png
4936    18523.png
2094    15839.png
8858    13948.png
          ...    
9667    17698.png
9217    12241.png
6611     9991.png
235     18639.png
8818    13908.png
Name: ImageName, Length: 6927, dtype: object


## Image functions

In [7]:
#Create a a function to add image according to the name given from the list

def GetImage(directory):
    images=[]
    for name in tqdm(directory, desc="Adding images"):
        image = cv2.imread("patch_images/"+name)
        image = Image.fromarray(image,'RGB')
        images.append(np.array(image))
    result = np.array(images)
    print("\ngetImage COMPLETED!")
    return result



In [8]:
# Get images ready by loading and preprocessing.
def load_real_samples(n_classes=10):
    (trainX, trainy), (_, _) = load_data()
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = (X - 127.5) / 127.5  # scale from [0,255] to [-1,1] as we will be using tanh activation. 
    print(X.shape, trainy.shape)
    return [X, trainy]


In [13]:
#Pick a subset of images. 
num_images = 120
def select_subset_images(dataset, n_samples=num_images, n_classes=10):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes) #Number of samples per class.
    for i in range(n_classes):
        X_with_class = X[y == i] # get all images for this class
        ix = randint(0, len(X_with_class), n_per_class) # choose random images for each class
        [X_list.append(X_with_class[j]) for j in ix] # add to list
        [y_list.append(i) for j in ix]
    return np.asarray(X_list), np.asarray(y_list) #Returns a list of 2 numpy arrays corresponding to X and Y

In [14]:
#Create a function to generate sample to fix the Imblance of the dataset
def GenerateSample(X,Y):
    ros = RandomOverSampler(random_state = 1)
    x, y = ros.fit_resample(X.values.reshape(-1,1), Y)
    x = x.flatten()
    return x,y

## ------------------------

In [15]:
dataset = GetImage(traindata['ImageName'])
# X, Y = select_subset_images(dataset)


Adding images: 100%|█████████████████████████████████████████████████████████████| 6927/6927 [00:02<00:00, 3032.45it/s]


getImage COMPLETED!


In [28]:
print(dataset.astype('float32'))
print( (dataset - 127.5) / 127.5 )

[[[[142.  66. 117.]
   [151.  75. 125.]
   [136.  55. 106.]
   ...
   [124.  42.  97.]
   [150.  64. 121.]
   [147.  66. 122.]]

  [[127.  55. 105.]
   [136.  65. 114.]
   [153.  74. 124.]
   ...
   [138.  53. 108.]
   [148.  57. 112.]
   [139.  57. 110.]]

  [[122.  51. 102.]
   [132.  63. 112.]
   [188. 109. 160.]
   ...
   [146.  61. 116.]
   [131.  39.  93.]
   [118.  35.  86.]]

  ...

  [[149.  63. 121.]
   [165.  80. 136.]
   [159.  72. 125.]
   ...
   [133.  46.  98.]
   [139.  65. 115.]
   [148.  72. 123.]]

  [[158.  70. 128.]
   [168.  84. 139.]
   [164.  79. 132.]
   ...
   [157.  72. 127.]
   [174.  97. 150.]
   [173.  97. 150.]]

  [[148.  59. 117.]
   [164.  79. 135.]
   [161.  77. 132.]
   ...
   [183. 101. 158.]
   [172.  94. 149.]
   [167.  90. 144.]]]


 [[[207. 175. 220.]
   [248. 214. 255.]
   [211. 177. 223.]
   ...
   [194. 156. 230.]
   [172. 130. 209.]
   [181. 142. 225.]]

  [[205. 175. 217.]
   [235. 203. 246.]
   [211. 177. 222.]
   ...
   [211. 176. 247.]
 

In [16]:
x2_train = train_task2['ImageName']
y2_train = train_task2['cellType']
print("Original Dataset:\n",y2_train.value_counts())

#Generate sample
x2_train, y2_train = GenerateSample(x2_train,y2_train)
print("Sampled Dataset:\n",y2_train.value_counts())
x2_train = GetImage(x2_train)

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

x2_val = val_task2['ImageName']
x2_val = GetImage(x2_val)

y2_val = val_task2['cellType']

x2_test,x2_val,y2_test,y2_val = train_test_split(x2_val, y2_val,
                                              test_size=0.5, random_state=9, stratify = y2_val)


Original Dataset:
 2    2855
1    1780
0    1322
3     970
Name: cellType, dtype: int64
Sampled Dataset:
 2    2855
3    2855
1    2855
0    2855
Name: cellType, dtype: int64


Adding images: 100%|███████████████████████████████████████████████████████████| 11420/11420 [00:02<00:00, 5688.91it/s]



getImage COMPLETED!


Adding images: 100%|█████████████████████████████████████████████████████████████| 2969/2969 [00:00<00:00, 5417.34it/s]


getImage COMPLETED!


In [17]:
print("TASK 2 SHAPE:")
print("x1 shape:", x2_train.shape)
print("y1 shape:", y2_train.shape)
print("VALIDATION SHAPE:")
print("x1 shape:", x2_val.shape)
print("y1 shape:", y2_val.shape)
print("TEST SHAPE:")
print("x1 shape:", x2_test.shape)
print("y1 shape:", y2_test.shape)

TASK 2 SHAPE:
x1 shape: (11420, 27, 27, 3)
y1 shape: (11420,)
VALIDATION SHAPE:
x1 shape: (1485, 27, 27, 3)
y1 shape: (1485,)
TEST SHAPE:
x1 shape: (1484, 27, 27, 3)
y1 shape: (1484,)


In [18]:
X2_train_scaled = x2_train / 255
X2_val_scaled = x2_val / 255
X2_test_scaled = x2_test / 255


In [19]:
data_augmentation = tf.keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical",input_shape=(27,27,3)),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

# 4. Models

## 4.1 Convolutional neural network

### 4.1.1 Baseline model

model = Sequential([
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    
    layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(l=0.01),padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(l=0.01),padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    
    layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(l=0.01),padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(l=0.01),padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.4),
    
    layers.Flatten(),
    layers.Dense(512, kernel_regularizer=l2(l=0.01), activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(512, kernel_regularizer=l2(l=0.01), activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(4, activation='softmax')
])

opt = Adam(lr=0.0005, beta_1=0.4, beta_2=0.444, decay=1.0e-6, amsgrad=True)
model.compile(optimizer=opt , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.001, patience=10, verbose=1, mode='auto'),
    tf.keras.callbacks.ModelCheckpoint(
    filepath="task2/task2_weights.{epoch:02d}-valloss.{val_loss:.2f}.h5",
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, patience=6, min_lr=0.5e-15) 
]

EPOCHS = 500
BATCH_SIZE = 32

import time
start_time = time.time()

with tf.device('/GPU:0'): 
    history = model.fit(X2_train_scaled, y2_train,
                    validation_data=(X2_val_scaled, y2_val), 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    verbose=1,
                    callbacks=my_callbacks)

print("---  Training time in seconds ---%s " % (time.time() - start_time))


scores = model.evaluate(X2_val_scaled, y2_val, verbose=1)
print('Validation loss:', scores[0])
print('Validation accuracy:', scores[1])
print('Max Validation accuracy:', max(history.history['accuracy']))

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

predict=model.predict(X2_test_scaled)
classes=np.argmax(predict,axis=1)

print(classification_report(y2_test, classes))


class_names = ["fibroblast","inflammatory","epithelial","others"]

import numpy as np
batch_predict = model.predict (X2_test_scaled)
plt.figure(figsize=(15, 15))

for i in range(9):
    ax = plt.subplot(3, 3, i+1)
    first_image = X2_test_scaled[i]
    first_label = int(y2_test.iloc[i])
    plt.imshow(first_image)
    confidence = round(100 * (np.max(batch_predict[i][0])), 2)
    actual_class= class_names[first_label]
    predicted_class = class_names[np.argmax(batch_predict[i])]
    
    plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
    plt.axis("off")
    

### 4.1.2 Hyperparameter tuning
#### I. Layers tuning

### 4.1.3 Final model

tf.config.experimental.list_physical_devices()

 model.load_weights("task2_weights.111-0.76.h5")